In [ ]:
from ipyprogressivis.widgets.scatterplot import Scatterplot
sc = Scatterplot()
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from progressivis.cluster import MBKMeans, MBKMeansFilter
from progressivis.io.api import SimpleCSVLoader
from progressivis.vis import MCScatterPlot
from progressivis.datasets import get_dataset
import pandas as pd
import numpy as np
import os.path
from progressivis.datasets import get_dataset
 
data = SimpleCSVLoader(get_dataset("bigfile_mvn"), throttle=1000, skipinitialspace=True, header=None)
n_clusters = 3
mbkmeans = MBKMeans(n_clusters=n_clusters, 
                    batch_size=100, tol=0.01, is_input=False)
mbkmeans.create_dependent_modules(data)
classes = []
for i in range(n_clusters):
    cname = f"k{i}"
    filt = MBKMeansFilter(i)
    filt.create_dependent_modules(mbkmeans, data, 'result')
    classes.append({'name': cname, 'x_column': '_0',
                    'y_column': '_1', 'sample': mbkmeans if i==0 else None,
                    'sample_slot': 'result',
                    'input_module': filt, 'input_slot': 'result'})
sp = MCScatterPlot(classes=classes)
sp.create_dependent_modules()
for i in range(n_clusters):
    cname = f"k{i}"
    sp[cname].min_value.result.update({'_0': -np.inf, '_1': -np.inf})
    sp[cname].max_value.result.update({'_0': np.inf, '_1': np.inf})

sp.move_point = mbkmeans.dep.moved_center # for inpula bt management
sc.link_module(sp)
sp.scheduler.task_start();

In [ ]:
sc